# Build a variational autoencoder

Based on the ability to generate UK car registration numbers, one can build a dataset of training and test data. With a suitable vectorizer, one can go ahead and make an autoencoder.

The decoder part of the variational encoder can then be used to generate new reg numbers.

### References

https://blog.keras.io/building-autoencoders-in-keras.html

https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73

In [1]:
import sys
import math
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import backend
from keras import optimizers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegDvlaVectorizer

### Data generation

In [2]:
n_train = 10000
n_val = 1000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegDvlaVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
val_strs = [g.get_reg() for _ in range(n_val)]
val_strs = [x for x in val_strs if x not in set(train_strs)]  # sanity check
val_vecs = np.array([v.vectorize(x) for x in val_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs) | set(val_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of validation samples = ' + str(len(val_strs)))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 150
actual number of validation samples = 1000
actual number of test samples = 1000


### Variational autoencoder

Design the architecture

In [3]:
latent_dim = 15  # 15 works better for a good model accuracy, 
                # but lower dimensions work better for directly using the latent space

### Define the encoder
inputs = keras.Input(shape=(vec_length,))
h = inputs
h = layers.Dense(vec_length, activation='relu')(h)
h = layers.Dense(50, activation='relu')(h)
h = layers.Dense(25, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = backend.random_normal(shape=(backend.shape(z_mean)[0], latent_dim),
                                    mean=0., stddev=0.1)
    return z_mean + backend.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

### Define the decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = latent_inputs
x = layers.Dense(25, activation='relu')(x)
x = layers.Dense(50, activation='relu')(x)
x = layers.Dense(vec_length, activation='relu')(x)
x = layers.Dense(vec_length, activation='sigmoid')(x)
outputs = x
decoder = keras.Model(latent_inputs, outputs, name='decoder')

outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

### Define the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= vec_length
kl_loss = 1 + z_log_sigma - backend.square(z_mean) - backend.exp(z_log_sigma)
kl_loss = backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

Do some training. The training below has a hard time optimizing when the latent dimension is less than 15, but for some of the techniques used later in the notebook, a much smaller latent dimension is required.

In [4]:
opt = optimizers.Adamax(learning_rate=0.004)  # larger learning rate for faster descent
vae.compile(optimizer=opt, loss='binary_crossentropy')
vae.fit(train_vecs, train_vecs,
        epochs=200,
        batch_size=8,
        shuffle=True,
        validation_data=(val_vecs, val_vecs))

opt = optimizers.Adamax(learning_rate=0.001)  # slower learning rate for approaching local min
vae.compile(optimizer=opt, loss='binary_crossentropy')
vae.fit(train_vecs, train_vecs,
        epochs=200,
        batch_size=8,
        shuffle=True,
        validation_data=(val_vecs, val_vecs))

Epoch 1/200
1250/1250 [==============================] - 1s 945us/step - loss: 28.4330 - val_loss: 26.7790
Epoch 2/200
1250/1250 [==============================] - 1s 824us/step - loss: 25.5594 - val_loss: 24.5344
Epoch 3/200
1250/1250 [==============================] - 1s 887us/step - loss: 23.4502 - val_loss: 22.9401
Epoch 4/200
1250/1250 [==============================] - 1s 913us/step - loss: 22.2339 - val_loss: 22.0361
Epoch 5/200
1250/1250 [==============================] - 1s 976us/step - loss: 21.5440 - val_loss: 21.5660
Epoch 6/200
1250/1250 [==============================] - 1s 847us/step - loss: 21.0753 - val_loss: 21.1477
Epoch 7/200
1250/1250 [==============================] - 1s 851us/step - loss: 20.6930 - val_loss: 20.8693
Epoch 8/200
1250/1250 [==============================] - 1s 858us/step - loss: 20.3107 - val_loss: 20.6205
Epoch 9/200
1250/1250 [==============================] - 1s 879us/step - loss: 19.8597 - val_loss: 20.1434
Epoch 10/200
1250/1250 [=============

1250/1250 [==============================] - 1s 958us/step - loss: 12.7736 - val_loss: 14.6084
Epoch 78/200
1250/1250 [==============================] - 1s 937us/step - loss: 12.7365 - val_loss: 14.4377
Epoch 79/200
1250/1250 [==============================] - 1s 987us/step - loss: 12.6633 - val_loss: 14.4074
Epoch 80/200
1250/1250 [==============================] - 1s 942us/step - loss: 12.5815 - val_loss: 14.3708
Epoch 81/200
1250/1250 [==============================] - 1s 963us/step - loss: 12.5090 - val_loss: 14.2576
Epoch 82/200
1250/1250 [==============================] - 1s 982us/step - loss: 12.4429 - val_loss: 13.9594
Epoch 83/200
1250/1250 [==============================] - 1s 987us/step - loss: 12.3865 - val_loss: 14.0191
Epoch 84/200
1250/1250 [==============================] - 1s 941us/step - loss: 12.2962 - val_loss: 14.0030
Epoch 85/200
1250/1250 [==============================] - 1s 936us/step - loss: 12.2670 - val_loss: 13.8561
Epoch 86/200
1250/1250 [=================

1250/1250 [==============================] - 1s 913us/step - loss: 9.7713 - val_loss: 11.6804
Epoch 153/200
1250/1250 [==============================] - 1s 883us/step - loss: 9.7279 - val_loss: 11.5876
Epoch 154/200
1250/1250 [==============================] - 1s 828us/step - loss: 9.7305 - val_loss: 11.7604
Epoch 155/200
1250/1250 [==============================] - 1s 900us/step - loss: 9.6619 - val_loss: 11.3276
Epoch 156/200
1250/1250 [==============================] - 1s 871us/step - loss: 9.6531 - val_loss: 11.4348
Epoch 157/200
1250/1250 [==============================] - 1s 881us/step - loss: 9.6221 - val_loss: 11.4589
Epoch 158/200
1250/1250 [==============================] - 1s 849us/step - loss: 9.6067 - val_loss: 11.5233
Epoch 159/200
1250/1250 [==============================] - 1s 900us/step - loss: 9.5301 - val_loss: 11.6295
Epoch 160/200
1250/1250 [==============================] - 1s 849us/step - loss: 9.5359 - val_loss: 11.4935
Epoch 161/200
1250/1250 [=================

Epoch 29/200
1250/1250 [==============================] - 1s 925us/step - loss: 6.5225 - val_loss: 8.2897
Epoch 30/200
1250/1250 [==============================] - 1s 1ms/step - loss: 6.5140 - val_loss: 8.2905
Epoch 31/200
1250/1250 [==============================] - 1s 1ms/step - loss: 6.5354 - val_loss: 8.1403
Epoch 32/200
1250/1250 [==============================] - 1s 971us/step - loss: 6.5046 - val_loss: 8.1704
Epoch 33/200
1250/1250 [==============================] - 1s 917us/step - loss: 6.5124 - val_loss: 8.1706
Epoch 34/200
1250/1250 [==============================] - 1s 905us/step - loss: 6.4671 - val_loss: 8.3638
Epoch 35/200
1250/1250 [==============================] - 1s 910us/step - loss: 6.4595 - val_loss: 8.1959
Epoch 36/200
1250/1250 [==============================] - 1s 865us/step - loss: 6.4675 - val_loss: 8.2230
Epoch 37/200
1250/1250 [==============================] - 1s 900us/step - loss: 6.4367 - val_loss: 8.2170
Epoch 38/200
1250/1250 [==========================

1250/1250 [==============================] - 1s 860us/step - loss: 5.8596 - val_loss: 7.5918
Epoch 107/200
1250/1250 [==============================] - 1s 975us/step - loss: 5.8214 - val_loss: 7.5390
Epoch 108/200
1250/1250 [==============================] - 1s 1ms/step - loss: 5.8203 - val_loss: 7.5481
Epoch 109/200
1250/1250 [==============================] - 1s 1ms/step - loss: 5.8252 - val_loss: 7.5357
Epoch 110/200
1250/1250 [==============================] - 2s 1ms/step - loss: 5.7966 - val_loss: 7.5676
Epoch 111/200
1250/1250 [==============================] - 1s 987us/step - loss: 5.7852 - val_loss: 7.4611
Epoch 112/200
1250/1250 [==============================] - 1s 982us/step - loss: 5.7918 - val_loss: 7.4901
Epoch 113/200
1250/1250 [==============================] - 1s 966us/step - loss: 5.7955 - val_loss: 7.4831
Epoch 114/200
1250/1250 [==============================] - 1s 1ms/step - loss: 5.7755 - val_loss: 7.6833
Epoch 115/200
1250/1250 [==============================] - 

1250/1250 [==============================] - 1s 895us/step - loss: 5.3678 - val_loss: 6.9287
Epoch 184/200
1250/1250 [==============================] - 1s 923us/step - loss: 5.3410 - val_loss: 6.9513
Epoch 185/200
1250/1250 [==============================] - 1s 913us/step - loss: 5.3520 - val_loss: 6.9816
Epoch 186/200
1250/1250 [==============================] - 1s 926us/step - loss: 5.3355 - val_loss: 6.9629
Epoch 187/200
1250/1250 [==============================] - 1s 913us/step - loss: 5.3420 - val_loss: 6.8922
Epoch 188/200
1250/1250 [==============================] - 1s 917us/step - loss: 5.3204 - val_loss: 6.9202
Epoch 189/200
1250/1250 [==============================] - 1s 908us/step - loss: 5.3230 - val_loss: 7.0155
Epoch 190/200
1250/1250 [==============================] - 1s 904us/step - loss: 5.3035 - val_loss: 6.7945
Epoch 191/200
1250/1250 [==============================] - 1s 933us/step - loss: 5.3028 - val_loss: 6.9190
Epoch 192/200
1250/1250 [==========================

Evaluation

In [5]:
decoded_regs = vae.predict(test_vecs)
print(test_strs[:10])
recovered = [v.recover(x) for x in decoded_regs]
print(recovered[:10])
acc = np.sum([x == y for x, y in zip(recovered, test_strs)]) / len(test_strs)
print('overall accuracy = ' + str(acc))
print('by digit accuracy:')
for i_digit in range(len(test_strs[0])):
    acc = np.sum([x[i_digit] == y[i_digit] for x, y in zip(recovered, test_strs)]) / len(test_strs)
    print('    ' + str(acc))

['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCX', 'SG47KFF', 'HB17WIY', 'CA89OXN', 'GT57YKA', 'CD77SJT']
overall accuracy = 0.517
by digit accuracy:
    0.991
    0.967
    0.992
    0.993
    0.688
    0.903
    0.882


### Use decoder as generator

By directly sampling in the latent space, and running through the decoder, one effectively has a data generator.

In [6]:
[v.recover(x) for x in decoder.predict(np.random.random((5, latent_dim)))]

['DH46CTU', 'HT45WOS', 'DO46WXL', 'DU55EPA', 'DK56WOQ']

One can do some sampling and look at some stats to see what kind of data comes out. For instance, the number of reg numbers with pair of integers below 50 and above 50

In [7]:
random_vectors = np.random.random((1000, latent_dim)) - 0.5
generated_regs = [v.recover(x) for x in decoder.predict(random_vectors)]
n_reg_below_50 = len([x for x in generated_regs if int(x[2:4]) < 50])
n_reg_above_50 = len([x for x in generated_regs if int(x[2:4]) >= 50])
print('reg numbers below 50 = ' + str(n_reg_below_50))
print('reg numbers above 50 = ' + str(n_reg_above_50))

reg numbers below 50 = 442
reg numbers above 50 = 558


### Avoid generating any training data

There is a chance that a random generation of data might produce one of the actual registration numbers used in training. However, any input registrations are represented as a probability distribution in the latent space; so one could require that any sample vector used in the latent space to be 'sufficiently far' down the tails of all the recorded distributions.

The distributions of the training data in the latent space are readily available:

In [8]:
# Get the distributions for each reg number in the data
all_strs = train_strs + val_strs + test_strs
all_vecs = np.array([v.vectorize(x) for x in all_strs])
means, log_sigmas, _ = encoder.predict(all_vecs)
distn_dict = {x: (y,z) for x, y, z in zip(all_strs, means, np.exp(log_sigmas))}  # dict<reg no: (mean, sigma)>
example_reg = list(distn_dict.keys())[0]
print('distribution for input data registration ' + str(example_reg))
print('    mu = ' + str(distn_dict[example_reg][0]))
print('    diagonal sigmas = ' + str(distn_dict[example_reg][1]))
# for a multivariate distribution ~ N(mean, Var) valued in the latent space, 
# mean is the first vector below, and Var is a diagonal matrix, the square root
# of the diagonal entries being given in the second vector.
# If the KL loss function pulls the distributions to be close to N(0,1)

distribution for input data registration YK66BIQ
    mu = [ 6.05341792e-01  1.34610564e-01  4.82149050e-03  1.87374011e-01
  6.64951280e-04 -2.46015787e-02 -1.18131995e-01  8.47651958e-01
  1.24904223e-01 -4.60249305e-01  4.73329067e-01 -5.29569853e-03
  1.75410032e-01  7.85022378e-01 -9.21758711e-02]
    diagonal sigmas = [0.5313385  0.44662854 0.98994035 0.6331824  0.994979   0.3902884
 0.9549835  0.4150101  0.39563507 0.33123204 0.4422034  0.99358207
 0.48427805 0.4332095  0.41187736]


One can work out a rough area in the latent space that the data vectors occupy. This could be important; certainly any point in the latent space can be used (since there is a distribution over the space), but how valid are samples from far outside the region covered by the 'heavy' parts of the obtained distributions?

In [9]:
all_means_mtr = np.array([x[0] for x in distn_dict.values()])
print('minimum of means over each latent dimension:')
print(np.min(all_means_mtr, axis=0))
print('maximum of means over each latent dimension:')
print(np.max(all_means_mtr, axis=0))

minimum of means over each latent dimension:
[-1.690712   -1.7782323  -0.03395703 -1.4910543  -0.03753981 -1.7040544
 -0.5108407  -1.7530599  -1.7487404  -1.5404701  -1.1142322  -0.0425394
 -1.4865733  -1.6822727  -1.7714614 ]
maximum of means over each latent dimension:
[1.6918695  1.9303555  0.02382918 1.2332906  0.01244401 1.698646
 0.74276054 1.8050513  1.8302792  1.6911187  1.1751331  0.01992849
 1.6836305  1.8916475  1.789688  ]


### Generating subsets of data with certain characteristics

As a more general example of 'avoid generating any training data', consider the case of 'avoid generating data with certain characteristics'. Since the latent space has a joint probability density function over it, this can be answered in a statistical manner. Consider the distributions of all data, that represent numbers below 50 (chosen since it is a characteristic that is to set up an experiment for). Recall all 7 digits are alpha numeric, with only the third and fourth digits being numeric

In [10]:
distns_below_50 = {k: v for k, v in distn_dict.items() if int(k[2:4]) < 50}
print('number of distributions with number below 50 = ' + str(len(distns_below_50)))

number of distributions with number below 50 = 6103


and use these to find only random vectors that are 'sufficiently far away' from the means of those reg numbers with numbers below 50. So the 'characteristic' chosen, will be for the number to be above 50; the reg numbers are of the form AAXXAAA, so the characteristic is that XX is greater than 50.

Employing a cheap-and-cheerful example of rejection sampling, and a cheap-and-cheerful example of statistical hypothesis testing, one can obtain the a set of random vectors that are 'sufficiently far' in the tails of the distributions belonging to those data vectors whose registration numbers are below 50.

In [26]:
# magnitude tuned so that it is roughly the volume covered by the means of the distributions
random_vectors = 2 * (np.random.random((1000, latent_dim)) - 0.5)
generated_regs = [v.recover(x) for x in decoder.predict(random_vectors)]
print('reg numbers below 50 = ' + str(len([x for x in generated_regs if int(x[2:4]) < 50])))
print('reg numbers above 50 = ' + str(len([x for x in generated_regs if int(x[2:4]) >= 50])))

reg numbers below 50 = 476
reg numbers above 50 = 524


In [28]:
# for each of the random vectors, find the smallest Mahalanobis distance from all the distributions with
# registration numeric part less than 50
means_mtr = np.array([mu for mu, sigma in distns_below_50.values()])
sigma_mtr = np.array([sigma for mu, sigma in distns_below_50.values()])
min_mahalanobis_dists = [np.min(np.linalg.norm((x - means_mtr) / sigma_mtr, axis=1))
                         for x in random_vectors]

# set a threshold on the Mahalanobis distance (which roughly corresponds to a hypothesis
# test significance level) to perform some crude rejection sampling
mahalanobis_threshold = 3.0
# I'm not convinced the latent_dim should be in the denominator here....
filtered_random_vectors = [x for x, y in zip(random_vectors, np.array(min_mahalanobis_dists)) 
                           if y > mahalanobis_threshold]
print('theoretical probability of sample from normal distribution having mahalanobis distance greater')
print('than mahalanobis threshold = ' +str(2*(1 - math.erf(0.5 * mahalanobis_threshold))))
print('number of filtered random vectors = ' + str(len(filtered_random_vectors)))

theoretical probability of sample from normal distribution having mahalanobis distance greater
than mahalanobis threshold = 0.06778970704937848
number of filtered random vectors = 106


In [29]:
if len(filtered_random_vectors) > 0:
    filtered_generated_regs = [v.recover(x) for x in decoder.predict(np.array(filtered_random_vectors))]
    filtered_n_reg_above_50 = len([x for x in filtered_generated_regs if int(x[2:4]) > 50])
    print('number of filtered reg numbers with numeric part above 50 = ' + str(filtered_n_reg_above_50))
    print('proportion of filtered reg numbers obeying chosen characteristic = '
          + str(filtered_n_reg_above_50/len(filtered_random_vectors)))
else:
    print('no filtered random vectors to work with')

number of filtered reg numbers with numeric part above 50 = 93
proportion of filtered reg numbers obeying chosen characteristic = 0.8773584905660378


Using the distributions to choose characteristics in the generated data works best when
- the latent dimension is low
- the random samples used in the latent space are not 'too large, i.e. not far away from the means of the distributions

The higher the dimension of the latent space, the greater care must be taken that the random samples used are in a suitable region of the latent space.

As a sanity check, consider the opposite chosen characterstic, and generating reg numbers where the numerics are below 50. This should yield similar results.

In [30]:
distns_above_50 = {k: v for k, v in distn_dict.items() if int(k[2:4]) >= 50}

means_mtr = np.array([mu for mu, sigma in distns_above_50.values()])
sigma_mtr = np.array([sigma for mu, sigma in distns_above_50.values()])
min_mahalanobis_dists = [np.min(np.linalg.norm((x - means_mtr) / sigma_mtr, axis=1))
                         for x in random_vectors]

mahalanobis_threshold = 3.0  
# filter on random samples that are 'far away' from test vectors that violate characteristic choice
filtered_random_vectors = [x for x, y in zip(random_vectors, np.array(min_mahalanobis_dists)) 
                           if y > mahalanobis_threshold]
filtered_generated_regs = [v.recover(x) for x in decoder.predict(np.array(filtered_random_vectors))]
filtered_n_reg_below_50 = len([x for x in filtered_generated_regs if int(x[2:4]) < 50])
print('number of filtered reg numbers with numeric part below 50 = ' + str(filtered_n_reg_below_50))
print('proportion of filtered reg numbers obeying chosen characteristic = '
        + str(filtered_n_reg_below_50/len(filtered_random_vectors)))

number of filtered reg numbers with numeric part below 50 = 108
proportion of filtered reg numbers obeying chosen characteristic = 0.8780487804878049
